# Development Notebook

In [1]:
EMBEDDINGS_DIRECTORY = './vstore' # directory of stored embeddings

In [ ]:
from dotenv import
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain.vectorstores import FAISS
from langchain_nomic.embeddings import NomicEmbeddings
from langchain.schema import Document
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    PromptTemplate
)

In [3]:
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
embedding_model = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
vectorstore = FAISS.load_local(
  'scripts/vstore', 
  embeddings = embedding_model, 
  allow_dangerous_deserialization = True
  )
retriever = vectorstore.as_retriever(search_kwargs={"k": 10, "fetch_k": 20})

In [5]:
retriever.get_relevant_documents("photos?") # Example query

/var/folders/xx/w3x7v3753jdc3rtcr5rvw8fw0000gn/T/ipykernel_52136/2417309861.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("photos?") # Example query
Embedding texts: 100%|██████████| 1/1 [00:00<00:00,  9.53inputs/s]


[Document(id='4f6118a3-2199-4c1b-be77-056d97aca848', metadata={'id': '2019.13.38', 'title': None, 'creator': 'Felice Beato', 'date': '1860s', 'medium': 'Albumen photoprint', 'format': '9 1/4" x 11 1/2"', 'subject': 'Graphic Arts-Photos', 'donor': 'Miami University Art Museum Purchase', 'citation': 'Felice Beato, “[Untitled],”Richard and Carole Cocks Art Museum at Miami University, accessed May 23, 2025,https://miamiuniversityartmuseum.omeka.net/items/show/20613.', 'tags': [], 'collection_link': None}, page_content='Title: Unknown\nCreator: Felice Beato\nDate: 1860s\nMedium: Albumen photoprint\nFormat: 9 1/4" x 11 1/2"\nSubject: Graphic Arts-Photos\nDescription: Looking under an arch to another building, presumably a temple'),
 Document(id='f75546a4-c7c7-4250-abfb-e467744756bd', metadata={'id': '2019.13.36', 'title': 'The Children of a Brother of Mr. Guikado', 'creator': 'Felice Beato', 'date': '1860s', 'medium': 'Albumen photoprint', 'format': '9" x 11 3/4"', 'subject': 'Graphic Arts-P

In [6]:
local_llm = OllamaLLM(model="llama3")
llm = ChatOllama(model="llama3", temperature=0)
llm_json_mode = ChatOllama(model="llama3", temperature=0, format="json")

In [7]:
system_temp = (
    "You are an expert art historian working for the Richard and Carole Cocks Art Museum at Miami University. "
    "You will be provided with a question and a set of contextual documents related to artworks, artists, or cultural history.\n\n"
    "Your task is to analyze the documents carefully and provide a thoughtful, accurate, and well-supported response based strictly on the information provided.\n\n"
    "Question: {question}\n\n"
    "Context: {context}\n\n"
    "If the answer cannot be found in the provided context, respond with: 'I don't know.'"
)


In [8]:
prompt = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["context", "question"],
                template=system_temp
            )
        )
    ]
)

In [9]:
docs = retriever.get_relevant_documents("What is the medium of the artwork?")

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 100.79inputs/s]


In [10]:
rag_chain = (
    RunnablePassthrough.assign(context=lambda x: "\n\n".join(d.page_content for d in retriever.get_relevant_documents(x["question"])))
    | prompt
    | llm
    | StrOutputParser()
)

# 4. Run the model (just provide the question!)
response = rag_chain.invoke({"question": "list the photographs and be sure to check if they are photos "})
print(response)

Embedding texts: 100%|██████████| 1/1 [00:00<00:00, 82.10inputs/s]


What a fascinating collection of photographs! As an art historian, I'm delighted to dive into these documents and provide some insights.

Based on the provided context, here are my observations:

1. The first set of photographs (Unknown, Temple of Asaxa Yedo, and The Temple of Choo-o-ng Kioto) appear to be taken by Felice Beato in the 1860s. These images depict various temples and architectural structures, likely from Japan or China during the Meiji period.
2. The second set of photographs (Elisabeth + Benght, 1981, and Lovers and Holy Child Nun, 1986) are attributed to Donna Ferrato. These images showcase a more contemporary style, with a focus on human subjects and everyday life.

Some specific observations:

* Felice Beato's photographs seem to be part of his extensive collection of Japanese and Chinese architecture, which was popular during the Meiji period.
* The Temple of Asaxa Yedo and The Temple of Choo-o-ng Kioto appear to be two distinct structures, possibly from different re

In [1]:
response = rag_chain.invoke({"question": "Tell me more about the long voyage home "})
print(response)

NameError: name 'rag_chain' is not defined